In [2]:
import cv2
import numpy as np
import os
import mediapipe as mp
import json

# --- ★変更点: 描画するパーツを選択するフラグ ---
# Trueに設定したパーツのランドマークが描画・保存されます
DRAW_FACE = False
DRAW_HANDS = True
DRAW_POSE = False 

# 設定
DATA_PATH = os.path.join(os.getcwd(), 'MP_Data_JSON')
actions = np.array(['ageru', 'understand', 'annsinnsuru' , 'heavy'])
no_videos = 10 #56
sequence_length = 30

# MediaPipe Holisticモデルの準備
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# 取得したい姿勢ランドマークのインデックスを指定
# DRAW_POSEがTrueの場合に、ここで指定したインデックスのランドマークのみが処理されます。
#DESIRED_POSE_LANDMARKS = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
# 顔と手は全てのランドマークを描画・保存対象としますが、
# 特定のインデックスのみにしたい場合は、上記のようにリストを作成して利用することも可能です。

def multiple_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    """
    指定されたフラグに基づいて、顔、手、ポーズのランドマークを描画する関数
    """
    # 顔のランドマークを描画 (DRAW_FACEがTrueの場合)
    if DRAW_FACE and results.face_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.face_landmarks,
            mp_holistic.FACEMESH_TESSELATION, # 顔のメッシュ
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(200, 200, 200), thickness=1, circle_radius=1) # メッシュを薄い灰色で描画
        )
        mp_drawing.draw_landmarks(
            image,
            results.face_landmarks,
            mp_holistic.FACEMESH_CONTOURS, # 顔の輪郭
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=1, circle_radius=1) # 輪郭を少し濃い灰色で描画
        )

    # 手のランドマークを描画 (DRAW_HANDSがTrueの場合)
    if DRAW_HANDS:
        # 左手
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2), # 点を赤色で描画
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 100, 100), thickness=2) # 線を薄い赤色で描画
            )
        # 右手
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2), # 点を青色で描画
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(100, 100, 255), thickness=2) # 線を薄い青色で描画
            )

    # 姿勢のランドマークを描画 (DRAW_POSEがTrueの場合)
    # この部分は元のコードのカスタム描画ロジックを維持
    if DRAW_POSE and results.pose_landmarks:
        image_height, image_width, _ = image.shape
        # 点の描画
        for idx in DESIRED_POSE_LANDMARKS:
            if idx < len(results.pose_landmarks.landmark):
                landmark = results.pose_landmarks.landmark[idx]
                if landmark.visibility < 0.1:
                    continue
                cx = int(landmark.x * image_width)
                cy = int(landmark.y * image_height)
                cv2.circle(image, (cx, cy), 3, (0, 255, 0), cv2.FILLED) # ポーズの点は緑色

        # 線の描画
        if mp_holistic.POSE_CONNECTIONS:
            for connection in mp_holistic.POSE_CONNECTIONS:
                start_idx, end_idx = connection
                if start_idx in DESIRED_POSE_LANDMARKS and end_idx in DESIRED_POSE_LANDMARKS:
                    if start_idx < len(results.pose_landmarks.landmark) and end_idx < len(results.pose_landmarks.landmark):
                        start_lm = results.pose_landmarks.landmark[start_idx]
                        end_lm = results.pose_landmarks.landmark[end_idx]
                        if start_lm.visibility < 0.1 or end_lm.visibility < 0.1:
                            continue
                        start_point = (int(start_lm.x * image_width), int(start_lm.y * image_height))
                        end_point = (int(end_lm.x * image_width), int(end_lm.y * image_height))
                        cv2.line(image, start_point, end_point, (51, 255, 51), 2)


def extract_keypoints(results):
    """
    指定されたフラグに基づいて、顔、手、ポーズのキーポイントを抽出する関数
    """
    keypoints_data = {}

    # 顔のランドマークを抽出 (DRAW_FACEがTrueの場合)
    if DRAW_FACE:
        if results.face_landmarks:
            face_landmarks_data = [{"id": i, "x": res.x, "y": res.y, "z": res.z} for i, res in enumerate(results.face_landmarks.landmark)]
            keypoints_data["face"] = face_landmarks_data
        else:
            keypoints_data["face"] = []
    
    # 左手のランドマークを抽出 (DRAW_HANDSがTrueの場合)
    if DRAW_HANDS:
        if results.left_hand_landmarks:
            left_hand_landmarks_data = [{"id": i, "x": res.x, "y": res.y, "z": res.z} for i, res in enumerate(results.left_hand_landmarks.landmark)]
            keypoints_data["left_hand"] = left_hand_landmarks_data
        else:
            keypoints_data["left_hand"] = []

    # 右手のランドマークを抽出 (DRAW_HANDSがTrueの場合)
    if DRAW_HANDS:
        if results.right_hand_landmarks:
            right_hand_landmarks_data = [{"id": i, "x": res.x, "y": res.y, "z": res.z} for i, res in enumerate(results.right_hand_landmarks.landmark)]
            keypoints_data["right_hand"] = right_hand_landmarks_data
        else:
            keypoints_data["right_hand"] = []

    # 姿勢のランドマークを抽出 (DRAW_POSEがTrueの場合)
    if DRAW_POSE:
        if results.pose_landmarks:
            pose_landmarks_data = []
            for i, res in enumerate(results.pose_landmarks.landmark):
                if i in DESIRED_POSE_LANDMARKS:
                    pose_landmarks_data.append({"id": i, "x": res.x, "y": res.y, "z": res.z, "visibility": res.visibility})
            keypoints_data["pose"] = pose_landmarks_data
        else:
            keypoints_data["pose"] = []
    
    return keypoints_data

# データの収集
cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)
cv2.resizeWindow('OpenCV Feed', (1280, 720))

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic_model:
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        if not os.path.exists(action_path):
            os.makedirs(action_path)

        for video_num in range(1, no_videos + 1):
            #video_file_path = os.path.join(r"C:\Users\admin\Downloads\syuwavideo", action + str(video_num) + '.mp4')
            video_file_path = os.path.join(r"D:\卒研手話\video\hikensya5", action + str(video_num) + '.mp4')
            cap = cv2.VideoCapture(video_file_path)

            video_data_save_path = os.path.join(action_path, str(video_num))
            if not os.path.exists(video_data_save_path):
                os.makedirs(video_data_save_path)

            print(f'Processing video: {video_file_path}')
            frames_collected_for_current_video = 0

            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                if not ret:
                    print(f"Warning: Video {video_file_path} ended before collecting {sequence_length} frames. Collected {frame_num} frames.")
                    break

                image, results = multiple_detection(frame, holistic_model)
                draw_styled_landmarks(image, results)

                cv2.putText(image, f'Collecting frames for {action} - Video {video_num} - Frame {frame_num + 1}/{sequence_length}',
                            (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                json_path = os.path.join(video_data_save_path, f'{frame_num}.json')
                with open(json_path, 'w') as f:
                    json.dump(keypoints, f, indent=4)

                frames_collected_for_current_video += 1

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
            cap.release()
            if frames_collected_for_current_video < sequence_length:
                print(f"Warning: For video {video_file_path}, only {frames_collected_for_current_video}/{sequence_length} frames were saved to {video_data_save_path}.")

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

Processing video: D:\卒研手話\video\hikensya5\ageru1.mp4
Processing video: D:\卒研手話\video\hikensya5\ageru2.mp4
Processing video: D:\卒研手話\video\hikensya5\ageru3.mp4
Processing video: D:\卒研手話\video\hikensya5\ageru4.mp4
Processing video: D:\卒研手話\video\hikensya5\ageru5.mp4
Processing video: D:\卒研手話\video\hikensya5\ageru6.mp4
Processing video: D:\卒研手話\video\hikensya5\ageru7.mp4
Processing video: D:\卒研手話\video\hikensya5\ageru8.mp4
Processing video: D:\卒研手話\video\hikensya5\ageru9.mp4
Processing video: D:\卒研手話\video\hikensya5\ageru10.mp4
Processing video: D:\卒研手話\video\hikensya5\understand1.mp4
Processing video: D:\卒研手話\video\hikensya5\understand2.mp4
Processing video: D:\卒研手話\video\hikensya5\understand3.mp4
Processing video: D:\卒研手話\video\hikensya5\understand4.mp4
Processing video: D:\卒研手話\video\hikensya5\understand5.mp4
Processing video: D:\卒研手話\video\hikensya5\understand6.mp4
Processing video: D:\卒研手話\video\hikensya5\understand7.mp4
Processing video: D:\卒研手話\video\hikensya5\understand8.mp4
Proce